In [ ]:
import numpy as np

from sklearn.covariance import GraphLasso, GraphLassoCV

# Graph inference

## Graph Lasso (fixed point in time, $t$)

In [ ]:
import numpy as np
def sparseinverse_cov(d):
    """Generate a covariance matrix with Sparse Inverse method.
    Sparse graph where each element depends only on a subset of others."""
    # W is a weight symmetric matrix.
    # It has -1 if two vertices are connected, 0 otherwise.
    # The diagonal is the sum of rows (or columns).
    W = np.zeros((d, d))
    for i in range(d):
        W[i, i+1:] = np.random.randint(-1, 1, d-i-1)
    W += W.T
    w = [abs(np.sum(vec)) for vec in W]
    laplacian = W + np.diag(w)
    return np.linalg.inv(laplacian + np.random.randn() ** 2 * np.eye(d))

In [ ]:
theta_t = sparseinverse_cov(3)

In [ ]:
theta_t

In [ ]:
from sklearn.covariance import GraphLasso, GraphLassoCV

In [ ]:
def random_covariance(n):
    Sinv = np.eye(n)
    idx = np.random.randint(2, size=n*n).reshape(n,n);
    for i in range(n):
        for j in range(n):
            Sinv[i,j] = idx[i,j]
    Sinv = Sinv + Sinv.T
    if np.min(np.linalg.eigh(Sinv)[0]) < 0:
        Sinv = Sinv + 1.1*np.abs(np.min(np.linalg.eigh(Sinv)[0]))*np.eye(n);

    S = np.linalg.inv(Sinv);
    return S, Sinv

In [ ]:
n_dim = 10
n_samples = 50
alpha=2*np.sqrt(np.log(n_dim) / n_samples)
print "alpha:", alpha

# true_covariance = sparseinverse_cov(n_dim)
# true_covariance /= np.diag(true_covariance)[0]
true_covariance, Sinv = random_covariance(n_dim)
X = np.random.multivariate_normal(np.zeros(n_dim), true_covariance, n_samples)

In [ ]:
gl = GraphLasso(mode='cd', alpha=alpha, verbose=0, max_iter=200)
gl = GraphLassoCV(verbose=0, max_iter=500, alphas=20)
gl.fit(X)

In [ ]:
from rgi import admm_covariance; reload(admm_covariance)

Cov, Z, hist = admm_covariance.covsel(X, .05,rho=2,alpha=alpha, verbose=0)

# print gl.error_norm(true_covariance)
# print np.linalg.norm(gl.covariance_ - true_covariance)

In [ ]:
print(np.linalg.norm(Sinv - Cov))

In [ ]:
print(np.linalg.norm(gl.precision_ - Sinv))

In [ ]:
Z

In [ ]:
np.set_printoptions(precision=1, suppress=True)

In [ ]:
gl.precision_

In [ ]:
Sinv

In [ ]:
Cov

In [ ]:
import pandas as pd
import numpy as np
from sklearn.covariance import GraphLasso, GraphLassoCV, empirical_covariance

df_x = pd.read_csv("/home/fede/projects_local/kdvs/", delimiter='\t', comment='#')
df_y = pd.read_csv("/home/fede/projects_local/petretto/data/fmf_labels.txt", delimiter='\t')

X = df_x.values
y = df_y.values.ravel()

In [ ]:
df_x.loc[:, df_x.columns.str.startswith("FMF_C")].values

In [ ]:
emp_cov = empirical_covariance(X[y=='A'], assume_centered=False)

In [ ]:
gl = GraphLasso(verbose=1, max_iter=200)
gl.fit(df_x.loc[:, df_x.columns.str.startswith("FMF_C")].values.T)

In [ ]:
n, d = 400, 10

X = np.random.randn(n, d)
beta = np.ones(d); beta[:d//2] = 0

y = X.dot(beta) + np.random.randn(n) * 0.1

In [ ]:
beta

In [ ]:
from sklearn.linear_model import Lasso

beta_lasso = Lasso(alpha=0.1).fit(X, y).coef_

In [ ]:
beta_lasso

In [ ]:
from rgi import admm_lasso as lasso; reload(lasso)

z = lasso.lasso(X, y, lamda=0.1, rho=1, alpha=1)

In [ ]:
z

In [ ]:
from rgi import admm_group_lasso as grouplasso; reload(grouplasso)

z_group = grouplasso.group_lasso(X, y, lamda=0.1, p=[[i] for i in range(10)],
                                 rho=1, alpha=1)

In [ ]:
z_group

In [ ]:
print "Error skle: %.3f" % np.linalg.norm(beta_lasso - beta)
print "Error admm: %.3f" % np.linalg.norm(z - beta)
print "Error admm_group: %.3f" % np.linalg.norm(z_group - beta)

In [ ]:
from rgi import admm_group_lasso_overlap as grouplassooverlap; reload(grouplassooverlap)

z_group = grouplassooverlap.GroupLassoOverlap(
        alpha=2, rho=2, groups=[[0,1,2,3,4], [3,4,5], [5,6,7,8,9]], verbose=0, max_iter=1000, tol=1e-6).fit(
X, y).coef_

In [ ]:
print "Error admm_group_overlap: %.3f" % np.linalg.norm(z_group - beta)

In [ ]:
%load_ext rpy2.ipython

In [ ]:
%%R
# install.packages("psych")
# install.packages("matrixcalc")
# install.packages("corpcor")
# install.packages("Matrix")
# install.packages("glasso")
install.packages("qgraph")

In [ ]:
%%R

library(psych)
library(matrixcalc)
library(Matrix)
library(glasso)
library(qgraph)
# E-step in the optimization algorithm:
Estep <- function(
  S, # Sample covariance
  Kcur, # Current estimate for K
  obs # Logical indicating observed
)
{
  stopifnot(require("Matrix"))
  stopifnot(isSymmetric(S))
  if (missing(Kcur))
  {
    if (missing(obs)) 
    {
      Kcur <- diag(nrow(S)) 
    } else 
    {
      Kcur <- diag(length(obs))
    }
  }
  stopifnot(isSymmetric(Kcur))
  if (missing(obs)) obs <- 1:nrow(Kcur) %in% 1:nrow(S)
  
  # To make life easier:
  H <- !obs
  O <- obs
  
  
  # Current estimate of S:
  library(corpcor)
  Scur <- pseudoinverse(Kcur)
  
  # Expected Sigma_OH:
  Sigma_OH <- S %*% pseudoinverse(Scur[O,O]) %*% Scur[O, H]
  #   Sigma_OH
  
  # Expected Sigma_H:
  Sigma_H <- Scur[H, H] - Scur[H,O] %*% pseudoinverse(Scur[O,O]) %*% Scur[O,H] + Scur[H,O] %*% pseudoinverse(Scur[O,O]) %*% S %*% pseudoinverse(Scur[O,O]) %*% Scur[O, H]
  
  # Construct expected sigma:
  Sigma_Exp <- rbind(cbind(S,Sigma_OH),cbind(t(Sigma_OH), Sigma_H))  
  
  return(Sigma_Exp)
}

# M-step in the optimization algorithm:
Mstep <- function(
  Sexp, # Expected full S
  obs, # Logica indiating oberved variables
  rho = 0,
  lambda
)
{
  if (!is.positive.definite(Sexp))
  {
    Sexp <- as.matrix(nearPD(Sexp)$mat)
    warning("Expected covariance matrix is not positive definite")
  }
  # Rho matrix:
  n <- nrow(Sexp)
  RhoMat <- matrix(rho, n, n)
  RhoMat[!obs,] <- 0
  RhoMat[,!obs] <- 0
  
  # Lambda:
  zeroes <- which(!lambda, arr.ind = TRUE)
  zeroes[,2] <- which(!obs)[zeroes[,2]]
  
  if (nrow(zeroes) > 0){
    K <- glasso(Sexp, RhoMat, penalize.diagonal=FALSE, zero = zeroes)$wi 
  } else {
    K <- glasso(Sexp, RhoMat, penalize.diagonal=FALSE)$wi
  }
  
  return(K)  
}

### Main lvglasso function
lvglasso <- function(
  S, # Sample cov
  nLatents, # Number of latents
  rho = 0, # Penalty
  thr = 1.0e-4, # Threshold for convergence (sum absolute diff)
  maxit = 1e4, # Maximum number of iterations
  lambda, # Logical matrix indicating the free factor loadings. Defaults to full TRUE matrix.
  ... # qgraph arguments
)
{
  if (missing(nLatents)){
    stop("'nLatents' must be specified")
  }
  
  nobs <- nrow(S)
  ntot <- nobs + nLatents
  
  if (missing(lambda) || is.null(lambda)){
    lambda <- matrix(TRUE, nobs, nLatents)
  }
  
  if (nrow(lambda) != nobs | ncol(lambda) != nLatents) stop("Dimensions of 'lambda' are wrong.")
  
  # PCA prior for K:
  efaRes <- principal(S, nfactors = nLatents)
  
#   # If sampleSizeis missing, set to 1000. Is only used for prior anyway.
# #   if (missing(sampleSize)){
#     sampleSize <- 1000
# #   }
# 
#   #   Get prior for K:
#   efaRes <- fa(S, n.obs= sampleSize, nfactors=nLatents)
  resid <- residuals(efaRes)
  class(resid) <- "matrix"
  load <- loadings(efaRes)
  class(load) <- "matrix"
  r <- efaRes$r.scores
  class(r) <- "matrix"
  r <- diag(diag(r))
#   
#   # Stupid nonanalytic way to get prior:
#   # Simulate N random variales:
#   #   eta <- rmvnorm(10000, rep(0, nLatents), r)
#   # 
#   #   # Simulate oserved scores:
#   #   Y <- eta %*% t(load) + rmvnorm(10000, rep(0, nobs), diag(diag(resid)
  # RAM FRAMEWORK:
  
  Sym <- rbind(cbind(diag( efaRes$uniquenesses),matrix(0,nobs,nLatents) ),cbind(t(matrix(0,nobs,nLatents) ),r))
  As <- matrix(0, ntot, ntot)
  if (nLatents > 0) As[1:nobs, (nobs+1):ntot] <- load
  
  Sigma <- solve(diag(ntot) - As) %*% Sym %*% t(solve(diag(ntot) - As))
  
  # Compute K:
# browser()
  K <- solve(cor2cov(cov2cor(Sigma),c(sqrt(diag(S)), rep(1, nLatents) )))
  #  K <- K  
#   K <- cov2cor(K)
      K=round(K,10)
  if (!is.positive.definite(K))
  {
    K <- as.matrix(nearPD(K)$mat)
#     warning("Expected covariance matrix is not positive definite")
  }

# browser()

  #   K <- matrix(-0.5,ntot,ntot)
  #   K[1:nobs,1:nobs] <- 0
  #   diag(K) <- 1
  
  #   K <- round(K,2)
  
  #   K <- EBICglasso(cov(cbind(Y,eta)), sampleSize)
  #   K <- as.matrix(forceSymmetric(cbind(rbind(pseudoinverse(resid),t(-load)), rbind(-load,pseudoinverse(r)))))
  
  # K <- as.matrix(forceSymmetric(cbind(rbind(pseudoinverse(resid),-0.5), rbind(rep(-0.5,nobs),1))))
  #   K <- as.matrix(forceSymmetric(cbind(rbind(diag(nrow(S)),t(-load)), rbind(-load,pseudoinverse(r)))))
  # browser()
  #   K <- matrix(-0.5,ntot,ntot)
  #   K[1:nobs,1:nobs] <- 0
  #   diag(K) <- 1
  #   obs <- c(rep(TRUE,nrow(S)), rep(FALSE,nLatents))
  # 
  #   K <- rbind(cbind(2*diag(nobs),-load),cbind(t(-load), 2*diag(ntot - nobs)))
  # rownames(K) <- colnames(K) <- NULL
  # 
  # diag(K) <- diag(K) - min(eigen(pseudoinverse(K))$values)
  
  # #   K[1:nobs,1:nobs] <- 0
  #   diag(K) <- 1
  obs <- c(rep(TRUE,nrow(S)), rep(FALSE,nLatents))
# 
# # Stupid prior:
# K <- matrix(0, ntot, ntot)
# diag(K) <- 1
# if (nLatents > 0) K[1:nobs, (nobs+1):ntot] <- K[ (nobs+1):ntot, 1:nobs] <- -1/nobs
  
  #   is.positive.definite(as.matrix(forceSymmetric(Estep(S, K, obs))))
  ### EM ###
it <- 1
Kold <- K
  repeat
  {
    Sexp <- as.matrix(forceSymmetric(Estep(S, K, obs)))
    #     Sexp <- cor2cov(cov2cor(Sexp),ifelse(obs,sqrt(diag(Sexp)),1))
    K <- as.matrix(forceSymmetric(Mstep(Sexp, obs, rho, lambda)))
    #     qgraph(wi2net(K), layout = "spring")
    
    # Check for convergence:
    if (sum(abs(cov2cor(pseudoinverse(Kold)[obs,obs]) - cov2cor(pseudoinverse(K)[obs,obs]))) < thr){
      break
    } else {
      it <- it + 1
      if (it > maxit){
        warning("Algorithm did not converge!")
        break
      } else {
        Kold <- K
      }
    }
  }

  if (!is.null(colnames(S)))
  {
    colnames(K) <- c(colnames(S),rep(paste0("F",seq_len(nLatents)))) 
  }
  
  if (is.null(rownames(S)))
  {
#       print(c(rep(paste0("N",seq_len(dim(K)[1] - nLatents))),rep(paste0("F",seq_len(nLatents)))) )
    rownames(K) <-c(rep(paste0("N",seq_len(dim(K)[1] - nLatents))),rep(paste0("F",seq_len(nLatents)))) 
  }

  # Partial correlations:
  pc <- qgraph:::wi2net(K)
  diag(pc) <- 1
  
rownames(pc) <- colnames(pc) <- rownames(K) <- colnames(K)

# Compute psychometric matrices:
Theta <- solve(K[obs, obs])
Lambda <- -Theta %*% K[obs,!obs]
Psi <- solve(K[!obs, !obs] - t(Lambda) %*% K[obs, obs] %*% Lambda)

# Return list mimics glasso:
Res <- list(
  w = pseudoinverse(K), # Estimated covariance matrix
  wi = K, # Estimated precision matrix
  pcor = pc, # Estimated partial correlation matrix
  observed = obs, # observed and latents indicator
  niter = it, # Number of iterations used in the algorithm
  lambda = Lambda,
  theta = Theta,
  psi = Psi
    )

  class(Res) <- "lvglasso"

  return(Res)
}

In [ ]:
from sklearn.covariance import empirical_covariance

covariance = empirical_covariance(X)

In [ ]:
%%R -i covariance -o Res

Res <- lvglasso(covariance, 3)